#<font color='#3dc1d3'>  CLINICAL TRIAL - Text extraction based on keyword Search

#Outline
---
# <font color='#3dc1d3'>  
1. Install required library, import and read Dataset

2. Longformer Transformer used to summerize the page content
    - Pretrained longformer model
    - Function to summerize the text for each row
    - Add all the summerized text into one cell
    - Merge new tranformed columns with the dataset
    
3. Save the cleaned dataset <br>


#1. Install Necesary packages

In [ ]:
!pip install PyPDF2
!pip install simpletransformers
!pip install PdfFileReader
!pip install re
!pip install transformers
!pip install tqdm
!pip install time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement PdfFileReader (from versions: none)
ERROR: No matching distribution found for PdfFileReader
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not fi

###Import Libraries

In [ ]:
import json
import pandas as pd
import urllib3
import io
from collections import defaultdict
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Read Dataset

In [ ]:
df1= pd.read_csv('/content/drive/MyDrive/Practicum/Final_folders/Submission/3.Data_Extraction_and_text_summerization_keyword_970_pending/Information_Extraction.csv')
df1.head(2)

,Unnamed: 0,Unnamed: 1,links,pages,Treatment,page_found,Protocol,Statistics,Procedure,adverse,...,Risk,Assessment,purpose,risk,Summary,objectives,ethical,Regulatory,Adverse events,Purpose
0,0,0,https://ClinicalTrials.gov/ProvidedDocs/74/NCT...,3,['Protocol \nChildren and adolescents with T1...,"[(1, 2), (1, 1)]",['Protocol \nChildren and adolescents with T1...,['The change of HbA1c values from baseline to...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,https://ClinicalTrials.gov/ProvidedDocs/94/NCT...,50,['30 \n Networks [Internet]. Pew Research Cent...,"[(1, 34), (1, 22), (4, 6), (1, 44), (1, 23), (...",['18 \n 7.1.1 Socio -demographic and health h...,['32 \n Interventions for Behavior Change in L...,['17 \n 6.9 Outcomes \n6.9.1 Primary outcome...,['5 \n mhealth interventions produced the high...,...,['30 \n Networks [Internet]. Pew Research Cent...,['34 \n \nAppendix 2 Food Literacy Questionna...,['40 \n 7. All study participants should at al...,['5 \n mhealth interventions produced the high...,NaN,NaN,NaN,NaN,NaN,NaN


#3.Longformer used to summerize the page content

##3.1 Pretrained longformer model

In [ ]:
from transformers import LongformerTokenizer, EncoderDecoderModel

# Load model and tokenizer
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16")
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096") 

You are using a model of type encoder_decoder to instantiate a model of type encoder-decoder. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at patrickvonplaten/longformer2roberta-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['encoder.embeddings.position_ids', 'decoder.roberta.pooler.dense.weight', 'decoder.roberta.pooler.dense.bias']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


##3.2 Function to summerize the text for each row 

In [ ]:
i=0
def text_summerise(text): 
  
  # Tokenize and summarize
  input_ids = tokenizer(text, return_tensors="pt").input_ids
  output_ids = model.generate(input_ids)

  # Get the summary from the output tokens
  summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  print(i+1)
  # Print summary
  return summary

In [ ]:
def func2():
  return " "

# Apply text summerization for each keyword text

In [ ]:
df1["objectives"] = df1.objectives.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["Objectives"] = df1.objectives.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

In [ ]:
df1["Purpose"] = df1.Purpose.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["purpose"] = df1.purpose.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

In [ ]:
df1["Statistics"] = df1.Statistics.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["treatment"] = df1.treatment.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

In [ ]:
df1["Treatment"] = df1.Treatment.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["assessment"] = df1.assessment.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

In [ ]:
df1["Assessment"] = df1.Assessment.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["adverse"] = df1.adverse.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

In [ ]:
df1["ethics"] = df1.ethics.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["Ethics"] = df1.Ethics.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

In [ ]:
df1["summary"] = df1.summary.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["Summary"] = df1.Summary.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

In [ ]:
df1["risk"] = df1.risk.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["Risk"] = df1.Risk.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

In [ ]:
df1["procedure"] = df1.procedure.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["Procedure"] = df1.Procedure.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

In [ ]:
df1["statistics"] = df1.statistics.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["Regulatory"] = df1.Regulatory.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

In [ ]:
df1["ethical"] = df1.ethical.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())
df1["Protocol"] = df1.Protocol.apply(lambda row: text_summerise(row) if pd.notnull(row) else  func2())

##3.3 Add all the summerized text into one cell

In [ ]:
df1["Summerised_pdf_based_on_keywords"] = df1[["objectives","Objectives", "Purpose","purpose","Statistics","treatment","Treatment","assessment","adverse","ethics","Ethics","summary","Summary","risk","Risk","procedure","Procedure","statistics","Regulatory","ethical","Protocol"]].apply("-".join, axis=1)

##3.4 Merge new tranformed columns with the dataset

In [ ]:
Final_Dataframe= pd.merge(df, df1, on="links")
Final_Dataframe

#4. Save the pipeline_part1_dataset

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Practicum/Final_folders/Submission/longformer/')
Final_Dataframe.to_csv('Full_dataset_pipeline_part_1.csv')
